In [77]:
from typing import Type
import numpy as np
import ufl
from dolfinx.fem import (Expression, Function, FunctionSpace, 
                         VectorFunctionSpace, dirichletbc, form, 
                         locate_dofs_topological)

from dolfinx.fem.petsc import (apply_lifting, assemble_matrix,
                               assemble_vector, set_bc)

from dolfinx.io import XDMFFile
from dolfinx.mesh import (CellType, GhostMode, create_box, locate_entities_boundary)

from mpi4py import MPI
from petsc4py import PETSc
from ufl import dx, grad, inner

from dolfinx import la, plot

import pyvista
pyvista.set_jupyter_backend('pythreejs');

D_TYPE = PETSc.ScalarType

/tmp/ipykernel_426/2353262281.py:21: PyVistaDeprecationWarning: `pythreejs` backend is deprecated and is planned for future removal.
  pyvista.set_jupyter_backend('pythreejs');


In [64]:
def build_nullspace(V: VectorFunctionSpace):
    """Build  PETSc nullspace for 3D elasticity"""
    
    # Create vector that span the nullspace
    bs = V.dofmap.index_map_bs;
    length0 = V.dofmap.index_map.size_local;
    length1 = length0 + V.dofmap.index_map.num_ghosts;
    basis = [np.zeros(bs * length1, dtype = D_TYPE) for i in range(6)];
    
    # Get dof indices for each subspace (x, y and z dofs)
    dofs = [V.sub(i).dofmap.list.array for i in range(3)];
    
    # Set the three translational rigid body modes
    for i in range(3):
        basis[i][dofs[i]] = 1.0;
    
    # Set the three rotational rigid body modes
    x = V.tabulate_dof_coordinates();
    dofs_block = V.dofmap.list.array;
    x0, x1, x2 = x[dofs_block, 0], x[dofs_block, 1], x[dofs_block, 2];
    
    basis[3][dofs[0]] = -x1;
    basis[3][dofs[1]] = x0;
    basis[4][dofs[0]] = x2;
    basis[4][dofs[2]] = -x0;
    basis[5][dofs[2]] = x1;
    basis[5][dofs[1]] = -x2;
    
    # Create PETSc Vec objects (excluding ghosts) and normalise
    basis_petsc = [PETSc.Vec().createWithArray(x[:bs*length0], bsize=3, comm=V.mesh.comm) for x in basis]
    la.orthonormalize(basis_petsc);
    assert la.is_orthonormal(basis_petsc);
    
    #Create and return a PETSc nullspace
    return PETSc.NullSpace().create(vectors=basis_petsc);

In [93]:
msh = create_box(MPI.COMM_WORLD, [np.array([0.0, 0.0, 0.0]),
                                  np.array([2.0, 1.0, 1.0])], [70, 70, 70],
                CellType.tetrahedron, GhostMode.shared_facet);
print("Number of DOF is", msh.geometry.x.shape[0]*msh.geometry.x.shape[1]);

Number of DOF is 1073733


In [94]:
ω, ρ = 300.0, 10.0;
x = ufl.SpatialCoordinate(msh);
f = ufl.as_vector((ρ * ω**2 * x[0], ρ * ω**2 * x[1], 0.0));

In [95]:
E = 1.0e9;
nu = 0.3;
μ = E / (2.0 * (1.0 + nu));
λ = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu));

def σ(v):
    """Return an expression for the stress σ given a displacement field"""
    return 2.0 * μ * ufl.sym(grad(v)) + λ * ufl.tr(ufl.sym(grad(v))) * ufl.Identity(len(v))

In [96]:
V = VectorFunctionSpace(msh, ("Lagrange", 1));
u, v = ufl.TrialFunction(V), ufl.TestFunction(V);
a = form(inner(σ(u), grad(v)) * dx);
L = form(inner(f, v) * dx);

In [97]:
facets = locate_entities_boundary(msh, dim=2,
                                  marker=lambda x: np.logical_or(np.isclose(x[0], 0.0),
                                                                 np.isclose(x[1], 1.0)));

bc = dirichletbc(np.zeros(3, dtype=D_TYPE), 
                 locate_dofs_topological(V, entity_dim=2, entities=facets), V=V);

In [98]:
A = assemble_matrix(a, bcs=[bc]);
A.assemble()

In [99]:
b = assemble_vector(L);
apply_lifting(b, [a], bcs=[[bc]]);
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE);
set_bc(b, [bc])

In [100]:
ns = build_nullspace(V);
A.setNearNullSpace(ns);

In [101]:
# set solver options
opts = PETSc.Options();
opts["ksp_type"] = "cg";
opts["ksp_rtol"] = 1.0e-10;
opts["pc_type"] = "gamg"; # geometric algebraic multigrid preconditioner

# Use Chebyshev smothing for multigrid
opts["mg_levels_ksp_type"] = "chebyshev";
opts["mg_levels_pc_type"] = "jacobi";

# Improve estimation of eigenvalues for Chebyshev smoothing
opts["mg_levels_esteig_ksp_type"] = "cg";
opts["mg_levels_ksp_chebyshev_esteig_steps"] = 20;

# Create PETSc Krylov solver and turn convergence monitoring on
solver = PETSc.KSP().create(msh.comm)
solver.setFromOptions()

# Set matrix operator
solver.setOperators(A)

In [102]:
uh = Function(V);

# Set a monitor, solve linear system and display the solver
# configuration
solver.setMonitor(lambda _, its, rnorm: print(f"Iteration: {its}, rel. residual: {rnorm}"));
solver.solve(b, uh.vector);
solver.view();

# Scatter forward the the solution ector to update ghost values
uh.x.scatter_forward()

Iteration: 0, rel. residual: 0.11762733250896973
Iteration: 1, rel. residual: 0.015186271012468474
Iteration: 2, rel. residual: 0.011617523250076865
Iteration: 3, rel. residual: 0.004499945103437487
Iteration: 4, rel. residual: 0.0011210865068802056
Iteration: 5, rel. residual: 0.0007418718517146908
Iteration: 6, rel. residual: 0.00043276162313169434
Iteration: 7, rel. residual: 0.0001668273287197931
Iteration: 8, rel. residual: 5.443276123187631e-05
Iteration: 9, rel. residual: 2.6413044650184187e-05
Iteration: 10, rel. residual: 1.2730435181881445e-05
Iteration: 11, rel. residual: 5.141086441520119e-06
Iteration: 12, rel. residual: 2.5701952096554293e-06
Iteration: 13, rel. residual: 1.2464332258123878e-06
Iteration: 14, rel. residual: 5.338736303158833e-07
Iteration: 15, rel. residual: 2.4929375118114083e-07
Iteration: 16, rel. residual: 1.2332859398220793e-07
Iteration: 17, rel. residual: 6.181632049407479e-08
Iteration: 18, rel. residual: 3.1438929046711394e-08
Iteration: 19, rel.

In [103]:
uh.name = "Deformation"

 
# visualize using pyvista
# to use it, ipycanvas and ipyvtklink py needs to be installed
# Just run in terminal:
# ~# pip install ipyvtklink ipycanvas
pyvista.start_xvfb()

# Create plotter and pyvista grid
p = pyvista.Plotter()
topology, cell_types, geometry = plot.create_vtk_mesh(msh)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# # Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=100)
actor_1 = p.add_mesh(warped, show_edges=True)
p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
   figure_as_array = p.screenshot("deformation.png")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…